In [69]:
# This notebook runs through the list of email addresses 
#  from the authors metadata and cleans them by 
#  fixing some commonly observed pathologies 

In [70]:
import pandas as pd
import numpy as np

In [71]:
#read the data file and extract non-null email entries
dat=pd.read_excel("./Authors.xlsx")
NonNullEmail=dat.loc[dat.email.notna()]
Emails=NonNullEmail.email

In [72]:
# Find the most common domain endings in the set.
# Since majority are entries, we can use thse to help us 
# split concatenated strings. Good domains have >1 characrter.
CommonEndings={}
for em in Emails:
    Ending=em.split('.')[-1]
    if(len(Ending)>1):
        if(not Ending in CommonEndings.keys()):
            CommonEndings[Ending]=1
        else:
            CommonEndings[Ending]+=1

# An ending is popular (and so probably real)
# if it has >5 use cases in the set.           
PopularEndings=[]
PopularCounts=[]
for key in CommonEndings.keys():
    if(CommonEndings[key]>5):
        PopularEndings.append("."+key)
        PopularCounts.append(CommonEndings[key])
endings=np.array(PopularEndings)

In [73]:
# I noticed from the dataset that we need to protect these two, 
# since they contain within them other good (more popular) domain
# strins
endings=endings[np.argsort(PopularCounts)][::-1]
endings=np.concatenate([[".int",".gov.uk"],endings])

print("Popular endings:")
print(endings)

Popular endings:
['.int' '.gov.uk' '.edu' '.com' '.cn' '.uk' '.de' '.fr' '.ca' '.org' '.au'
 '.tw' '.jp' '.gov' '.it' '.nl' '.kr' '.hk' '.ch' '.es' '.sg' '.br' '.se'
 '.sa' '.fi' '.net' '.be' '.pl' '.nz' '.at' '.no' '.za' '.ie' '.il' '.ir'
 '.mil' '.eu' '.ru' '.in' '.dk' '.th' '.cat' '.int' '.gr' '.pt' '.cz'
 '.ar' '.my' '.cl' '.qa' '.mx' '.co' '.si' '.com†']


In [74]:
# This function fixes two pathologies:
#  1) Bad characters or strings jammed into the email address - common ones
#       picked out by observation of the list;
#
#  2) Valid email concatenated at end with more text, identified via
#       noting occurence of popular domain name in the string.
#
#   [We cannot fix concatenated at beginning, since string jammed on front is
#   still a valid formatted address, in most cases.]

def FixIt(em):
    # Forbidden word /symbol cleaning
    forbidden=['author','emailaddress','correspondingauthor','telephone','<','>','*-','*',':','†']
    Changed=False
    for forb in forbidden:
        if(forb in em):
            splits=em.split(forb)
            for s in splits:
                #after split, the part contianing 
                #  an @ is an email address
                if "@" in s:   
                    em=s
                    
    # popular domain name based cleaning        
    for ending in endings:
        found=(em.find(ending))
        wheresat=em.find("@")

        if((found>0) and ((found+len(ending))<=len(em)) and (found > wheresat)):
            em=em[0:found+len(ending)]
            break
    
    return em


In [75]:
# Sample 100 for smell test
for em in Emails[0:100]:
    em2=FixIt(em)
    if(em2!=em):
        print(em + " -> " + em2)

jenny.gravel@daf.qld.gov.auj.l.g. -> jenny.gravel@daf.qld.gov.au
ssawyer@colorado.edusls -> ssawyer@colorado.edu
*ssawyer@colorado.edu -> ssawyer@colorado.edu
emma.l.walton@ntnu.noe.l.w. -> emma.l.walton@ntnu.no
*claudio.afonso@ars.usda.gov -> claudio.afonso@ars.usda.gov
debuysscherb@niaid.nih.govb.l.d. -> debuysscherb@niaid.nih.gov
vpopov@utmb.eduv.l.p. -> vpopov@utmb.edu
cbailey2@gmu.educ.l.b. -> cbailey2@gmu.edu
*east@izw-berlin.de -> east@izw-berlin.de
amylkistler*-amy@derisilab.ucsf.edu -> amy@derisilab.ucsf.edu
martinlhibberd-hibberdml@gis.a-star.edu.sg -> martinlhibberd-hibberdml@gis.a-star.edu
p.molyneaux@imperial.ac.ukp.l.m. -> p.molyneaux@imperial.ac.uk
sarahll@unimelb.edu.au -> sarahll@unimelb.edu
suzanne.epstein@fda.hhs.gov¤a -> suzanne.epstein@fda.hhs.gov
michelle.baker@csiro.aum.l.b. -> michelle.baker@csiro.au
brian_mark@umanitoba.cablm -> brian_mark@umanitoba.ca
grant.hughes@lstmed.ac.uk*correspondence:sanchitabhadra@utexas.edu -> sanchitabhadra@utexas.edu
ksarachan@alba

In [76]:
#Now go ahead and apply it to the dataset
WithEmails=dat.loc[dat.email.notna()]

def FixEmail(row):
    newem=FixIt(row.email)
    if(newem!=row.email):
        row.rawemail=row.email
        row.email=newem
    return row

WithEmails['originalemail']=WithEmails.email
WithEmails=WithEmails.apply(FixEmail,axis='columns')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [77]:
# Save it to a file
WithEmails.to_excel("CleanedEmails.xlsx")